In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent  # point back to the root of the project
context = load_context(proj_path)


In [ ]:
df = context.catalog.load('projections')
df.head()

In [ ]:
df.query('position == "rb"')

In [ ]:
import pandas as pd


In [ ]:
pd.read_html('https://www.pro-football-reference.com/years/2019/passing.htm','Passing Table')